ref: https://blog.csdn.net/qq_45672807/article/details/123294954

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import shutil

import time

In [2]:
# rec_time_to_date={
#     '2019-11-05--12:27:11':{64:'051119',177:'061119',291:'071119',404:'081119',457:'091119'},
#     '2019-11-11--11:34:50':{37:'2019-11-11--11:34:50'},
#     '2019-11-15--12:26:27':{66:'151119',76:'161119'},
#     '2019-11-22--11:20:15':{24:'2019-11-22--11:20:15'},
#     '2019-11-27--10:24:59':{84:'271119',199:'281119',314:'291119',429:'301119',544:'011219',585:'021219a'},
#     '2019-12-02--12:45:38':{63:'021219b',178:'031219',293:'041219',342:'051219'},
#     '2019-12-07--16:44:18':{28:'071219',144:'081219',259:'091219',297:'101219a'},
#     '2019-12-10--11:52:53':{71:'101219b',110:'111219'},
#  }

In [3]:
# res_dataset_dir='/home/pigs/pigs-behaviours/results_dataset'
# rec_times=[dd for dd in os.listdir(res_dataset_dir) if os.path.isdir(os.path.join(res_dataset_dir,dd))]
# rec_times.sort()
# rec_times=[]

In [4]:
root_dir='/home/pigs/SlowFast_models/test_data_prep'

frame_dir=os.path.join(root_dir,'AVA','frames')
if os.path.isdir(frame_dir):
    shutil.rmtree(frame_dir)
os.makedirs(frame_dir,exist_ok=True)

frame_list_dir=os.path.join(root_dir,'AVA','frame_lists')
if os.path.isdir(frame_list_dir):
    shutil.rmtree(frame_list_dir)
os.makedirs(frame_list_dir,exist_ok=True)

video_dir=os.path.join(root_dir,'AVA','videos')
if os.path.isdir(video_dir):
    shutil.rmtree(video_dir)
os.makedirs(video_dir,exist_ok=True)

video_dir=os.path.join(root_dir,'AVA','annotations')
if os.path.isdir(video_dir):
    shutil.rmtree(video_dir)
os.makedirs(video_dir,exist_ok=True)
# train_rec_times={'2019-11-22--11:20:15','2019-12-07--16:44:18'}

test_videos=[vi for vi in os.listdir('/home/pigs/SlowFast_models/test_data_prep/test_video') if '.mp4' in vi]

In [5]:
csvs={'train':[],'val':[]}
detect_csvs={'train':[],'val':[]}
bad_csvs={'train':[],'val':[]}
frame_lists={'train':['original_vido_id video_id frame_id path labels'],
             'val':['original_vido_id video_id frame_id path labels']}
# video_id=-1
start_time=time.time()
# for rec_time in rec_times:
    # if rec_time in train_rec_times:
    #     kind='val'
    # else:
    #     kind='train'
for vi in test_videos:

    if vi=='PIGS021219a_000545.mp4':
        kind='val'
    else:
        kind='train'

    # time_indices=sorted([tt for tt in os.listdir(os.path.join(res_dataset_dir,rec_time)) if os.path.isdir(os.path.join(res_dataset_dir,rec_time,tt))])
    # for time_index in time_indices: # e.g. 00063
    
    track_info =np.load(os.path.join(root_dir,'test_video',vi.split('.')[0],'tracklets.npy'),allow_pickle=True) # see gen_YOLO_data.ipynb for explanation
    frame_rate=track_info[0]
    resolution=track_info[2]

    time_now=time.time()
    print('='*40)
    print('Working on',vi)
    print('time now =',time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))
    print('time_taken =',round(time_now-start_time,2),'s')
    start_time=time_now

    track=np.load(os.path.join(root_dir,'test_video',vi.split('.')[0],'behaviour_15.npy'),allow_pickle=True)

    # # find the corresponding PIGS_data_dir (e.g. PIGS031219)
    # #
    # key_periods=sorted(list(rec_time_to_date[rec_time].keys()))
    # period_ind=0
    # time_index_val=int(time_index)
    # while time_index_val>key_periods[period_ind]:
    #     period_ind+=1
    # PIGS_date_dir=rec_time_to_date[rec_time][key_periods[period_ind]]
    # if '--' not in PIGS_date_dir:
    #     PIGS_date_dir='PIGS'+PIGS_date_dir
    # if PIGS_date_dir in {'PIGS011219'}:
    #     continue


    # # collect info on bad frames
    # #
    # bad_frames=set()
    # for i in range(len(track)): # loop over pig index
    #     for m in range(len(track[i])): # loop over the frame

    #         seconds_cnt=int(m//30+1)

    #         if seconds_cnt in bad_frames:
    #             continue

    #         if m>0 and m%30==29 and track[i][m][5]==2:
    #             bad_frames.add(seconds_cnt)
    #             bad_csvs[kind].append(PIGS_date_dir+'_'+time_index+',' +str(seconds_cnt))


    # prepare the directory to store the frame data
    frame_dir=os.path.join(root_dir,'AVA','frames',vi.split('.')[0])
    if not os.path.isdir(frame_dir):
        os.mkdir(frame_dir)

    # extract the frame to the frame_lists
    #
    # pig_video_path=os.path.join('/home/pigs/pigs-behaviours',PIGS_date_dir,time_index,'color.mp4')
    # # e.g. '/home/pigs/pigs-behaviours/PIGS051119/000000/color.mp4'
    # if not os.path.isfile(pig_video_path):
    #     continue
    
    # shutil.copy(pig_video_path,video_dir)
    # os.rename(os.path.join(video_dir,'color.mp4'),os.path.join(video_dir,PIGS_date_dir+'_'+time_index+'.mp4'))

    # video_id+=1

    pig_video_path=os.path.join(root_dir,'test_video',vi)
    cap = cv2.VideoCapture(pig_video_path)

    frame_cnt=0
    while (True):
        ret, frame = cap.read() 
        if ret:

            frame_dst_file=os.path.join(root_dir,'AVA','frames',vi.split('.')[0],vi.split('.')[0]+'_'+str(frame_cnt)+'.jpg')
            cv2.imwrite(frame_dst_file, frame)

            content=vi.split('.')[0]+' ' # original_vido_id
            content+= str(0)+' '#str(video_id)+' ' # video_id
            content+= str(frame_cnt)+' ' # frame_id
            content+= frame_dst_file+' ' # path
            content+= '""' # labels
            frame_lists[kind].append(content)

            frame_cnt+=1
            
        else: 
            break

    cv2.destroyAllWindows() 
    cap.release() 

    # extract the box and action label info
    #
    all_boxes=dict() # all_boxes[frame_ind][pig_ind]=csv
    detect_boxes=dict()

    seconds_cnt=0
    for i in range(len(track)): # loop over pig index
        for m in range(len(track[i])): # loop over the frame

            # if m in bad_frames:
            #     continue

            if m==0 or not m%30==29:
                continue

            seconds_cnt+=1

            csv=''
            detect_csv=''

            # (1st col) 视频数据的名称
            csv+=vi.split('.')[0]+','#PIGS_date_dir+'_'+time_index+',' 
            detect_csv+=vi.split('.')[0]+','#PIGS_date_dir+'_'+time_index+',' 

            # (2nd col) 开始的秒数
            csv+=str(seconds_cnt)+','
            detect_csv+=str(seconds_cnt)+','

            # (3-6th col) 框的四角坐标
            x0,x1=track[i][m][1]/resolution[0],track[i][m][3]/resolution[0]
            y0,y1=track[i][m][2]/resolution[1],track[i][m][4]/resolution[1]
            csv+=str(x0)+','+str(y1)+','+str(x1)+','+str(y0)+',' #lower left and upper right
            detect_csv+=str(x0)+','+str(y1)+','+str(x1)+','+str(y0)+',' #lower left and upper right

            # (7th col) 行为标签
            action_code=track[i][m][13]
            if action_code==101:
                action_code=7
            elif action_code==102:
                action_code=8
            action_code+=1 # to match those in the .pbtxt file # Oli 
            csv+=str(int(action_code))+','
            detect_csv+=str(int(action_code))+','

            # (8th col) 身份ID
            csv+=str(i)

            # (8th col) accuracy for the detection box
            detect_csv+=str(0.99)

            # store the csv info
            if seconds_cnt not in all_boxes:
                all_boxes[seconds_cnt]={i:csv}
                detect_boxes[seconds_cnt]={i:detect_csv}
            else:
                all_boxes[seconds_cnt][i]=csv # all_boxes[frame_ind][pig_ind]
                detect_boxes[seconds_cnt][i]=detect_csv
    
    csvs[kind].extend([all_boxes[m][i] for m in all_boxes for i in all_boxes[m]])
    detect_csvs[kind].extend([detect_boxes[m][i] for m in all_boxes for i in all_boxes[m]])

for kind in ['train','val']:
    save_csv_path=os.path.join(root_dir,'AVA','annotations','ava_'+kind+'_v2.2.csv')
    save_csv=open(save_csv_path,'w')
    save_csv.write('\n'.join(csvs[kind]))
    save_csv.close()

    save_detect_csv_path=os.path.join(root_dir,'AVA','annotations','ava_detection_'+kind+'_v2.2.csv')
    save_detect_csv=open(save_detect_csv_path,'w')
    save_detect_csv.write('\n'.join(detect_csvs[kind]))
    save_detect_csv.close()

    # save_bad_csv_path=os.path.join(root_dir,'AVA','annotations','ava_'+kind+'_excluded_timestamps.csv')
    # save_bad_csv=open(save_bad_csv_path,'w')
    # save_bad_csv.write('\n'.join(bad_csvs[kind]))
    # save_bad_csv.close()

    save_frame_list_path=os.path.join(root_dir,'AVA','frame_lists',kind+'.csv')
    save_frame_list=open(save_frame_list_path,'w')
    save_frame_list.write('\n'.join(frame_lists[kind]))
    save_frame_list.close()

del csvs
del bad_csvs
del frame_lists

# 以ava_train_v2.2.csv示例...

#  track[i] --> array with Mx14 dimension 
#  (M = tracklet length, i.e. the ith pig and the Mth trancklet) where the 14 values are in order 
#  (for track i and frame m of the track):

# e.g. track[0][10] : for the 0th pig and the its 10th tracklet

#  track[i][m][0] --> frameID
#  track[i][m][1] --> bbox_X_min
#  track[i][m][2] --> bbox_Y_min
#  track[i][m][3] --> bbox_X_max
#  track[i][m][4] --> bbox_Y_max
#  track[i][m][5] --> tracklet_state: 0 = new tracklet, 1 = updated tracklet, 2 = wrong tracklet or no available detection
#  track[i][m][6] --> temporal_analysis, i.e. mean absolute difference in terms of bounding box center pixel position with regard to the previous F (15, 20 or 25) frames
#  track[i][m][7] --> theta, i.e. angle of orientation of the pig
#  track[i][m][8] --> center_of_mass_Xs
#  track[i][m][9] --> center_of_mass_Y
#  track[i][m][10] --> length of the pig computed with moments
#  track[i][m][11] --> width of the pig computed with moments
#  track[i][m][12] --> zone in the pen where the center of mass of the pig is seen (0 = other, 1 = feeder, 2 = water bottom left, 3 = water bottom right, 4 = playing with yellow toy)
#  track[i][m][13] --> behavior code (0 = unknown, 1 = not moving, 2 = moving, 3 = running, 4 = eating, 5 = drinking, 6 = playing, 101 = standing, 102 = lying)

Working on PIGS021219a_000545.mp4
time now = 2024-12-05 15:51:49
time_taken = 0.03 s
Working on 2019-11-11--11:34:50_000000.mp4
time now = 2024-12-05 15:51:53
time_taken = 4.2 s
